<a href="https://colab.research.google.com/github/pulkitdixit/cnn_CIFAR10/blob/master/CIFAR100_Pytorch_GPU_GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [251]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import time

In [0]:
root = 'gdrive/My Drive/Google Colab/'
batch_size = 100
learn_rate = 0.001
scheduler_step_size = 5
scheduler_gamma = 0.5
num_epochs = 50

In [0]:
transform_train = transforms.Compose([transforms.RandomRotation(10),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()
                                     ])

In [0]:
train_dataset = torchvision.datasets.CIFAR10(root = root, train=True, transform=transform_train, download=False)
test_dataset = torchvision.datasets.CIFAR10(root = root, train=False, transform=transform_train, download=False)

In [0]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = 4, stride = 1, padding = 2),
            nn.ReLU()) #output image = 64*33*33
        self.layer1_bn = nn.BatchNorm2d(64)
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size = 4, stride = 1, padding = 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)) #output image = 64*17*17
        
        self.drop_out = nn.Dropout()
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size = 4, stride = 1, padding = 2),
            nn.ReLU()) #output image = 64*18*18
        self.layer3_bn = nn.BatchNorm2d(64)
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size = 4, stride = 1, padding = 2),
            nn.ReLU()) #output image = 64*19*19
        self.layer4_bn = nn.BatchNorm2d(64)
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size = 4, stride = 1, padding = 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)) #output image = 64*10*10
        self.layer5_bn = nn.BatchNorm2d(64)
        
        self.drop_out2 = nn.Dropout()
        
        self.layer6 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 0),
            nn.ReLU()) #output image = 64*8*8
        
        self.drop_out3 = nn.Dropout()
        
        self.layer7 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 0),
            nn.ReLU()) #output image = 64*6*6
        self.layer7_bn = nn.BatchNorm2d(64)
        
        self.layer8 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 0),
            nn.ReLU()) #output image = 64*4*4
        self.layer8_bn = nn.BatchNorm2d(64)
        
        self.drop_out4 = nn.Dropout()
        
        self.linear1 = nn.Linear(4*4*64, 500)
        self.linear2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer1_bn(x)
        
        x = self.layer2(x)
        
        x = self.drop_out(x)
        
        x = self.layer3(x)
        x = self.layer3_bn(x)
        
        x = self.layer4(x)
        x = self.layer4_bn(x)
        
        x = self.layer5(x)
        x = self.layer5_bn(x)
        
        x = self.drop_out2(x)
        
        x = self.layer6(x)
        
        x = self.drop_out3(x)
        
        x = self.layer7(x)
        x = self.layer7_bn(x)
        
        x = self.layer7(x)
        x = self.layer7_bn(x)
        
        x = self.drop_out4(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.linear1(x)
        
        x = self.linear2(x)
        
        return(x)

In [0]:
model = ConvNet()

#Enable GPU:
use_cuda = True
if use_cuda and torch.cuda.is_available():
    model.cuda()

In [0]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), 
                                lr = learn_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                            step_size = scheduler_step_size, 
                                            gamma = scheduler_gamma)

In [260]:
for epochs in range(num_epochs):
    scheduler.step()
    correct = 0
    total = 0
    print('Current epoch: \t\t', epochs+1, '/', num_epochs)
    #print('--------------------------------------------------')
    for images, labels in train_loader:
        #images = images.reshape(-1, 16*16)
        images = images
        labels = labels
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total = total + labels.size(0)
        correct = correct + (predicted == labels).sum().item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_acc = correct/total
    print('Training accuracy: \t', train_acc)
    #print('--------------------------------------------------')
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images
            labels = labels
            if use_cuda and torch.cuda.is_available():
              images = images.cuda()
              labels = labels.cuda()
        
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total = total + labels.size(0)
            correct = correct + (predicted == labels).sum().item()
    test_acc = correct/total
    print('Test Accuracy: \t\t', test_acc)
    print('**************************************************')
    model.train()

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Current epoch: 		 1 / 50
Training accuracy: 	 0.39964
Test Accuracy: 		 0.4838
**************************************************
Current epoch: 		 2 / 50
Training accuracy: 	 0.5558
Test Accuracy: 		 0.565
**************************************************
Current epoch: 		 3 / 50
Training accuracy: 	 0.62748
Test Accuracy: 		 0.62
**************************************************
Current epoch: 		 4 / 50
Training accuracy: 	 0.66916
Test Accuracy: 		 0.6539
**************************************************
Current epoch: 		 5 / 50
Training accuracy: 	 0.71918
Test Accuracy: 		 0.7298
**************************************************
Current epoch: 		 6 / 50
Training accuracy: 	 0.73392
Test Accuracy: 		 0.7333
**************************************************
Current epoch: 		 7 / 50
Training accuracy: 	 0.74664
Test Accuracy: 		 0.7498
**************************************************
Current epoch: 		 8 / 50
Training accuracy: 	 0.7561
Test Accuracy: 		 0.7646
****************